In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
mykey=os.getenv("openai_api_key")

In [5]:
print(mykey)

sk-proj-Mwk9fvaiNekvLycn0QJ6T3BlbkFJJwQVcJma2dhw5Xy4rPF3


In [6]:
llm = ChatOpenAI(openai_api_key=mykey, model_name="gpt-3.5-turbo", temperature=0.3)

c:\Users\venka\Anaconda3\envs\openaidemo\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000018291B70880>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000018292B50700>, temperature=0.3, openai_api_key='sk-proj-Mwk9fvaiNekvLycn0QJ6T3BlbkFJJwQVcJma2dhw5Xy4rPF3', openai_proxy='')

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""


In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [12]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [13]:
quiz_chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000018291B70880>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000018292B50700>, temperature=0.3, openai_api_key='sk-proj-Mwk9fvaiNekvLycn0QJ6T3BlbkFJJwQVcJma2dhw5Xy4rPF3', openai_proxy=''), output_key='quiz')

In [14]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [15]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [16]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [17]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [18]:
file_path=r"C:\Users\venka\OneDrive - Northeastern University\My Company\GitHub\master\LLM\projects\mcqgenerator\data.txt"

In [19]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [20]:
print(TEXT)

Deep learning is the subset of machine learning methods based on neural networks with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]

Deep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]

Early forms of neural networks were inspired by information processing and distributed communication nodes in biological systems, in particular the human brain. However, current neural networks do not intend to model the brain functi

In [21]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:

NUMBER=5 
SUBJECT="Deep Learning"
TONE="simple"

In [23]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\venka\Anaconda3\envs\openaidemo\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Deep learning is the subset of machine learning methods based on neural networks with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]

Deep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]

Early forms of neural networks were inspired by information processing and distributed communication nodes in biological syst

In [24]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1634
Prompt Tokens:880
Completion Tokens:754
Total Cost:0.0028280000000000002


In [45]:
response

{'text': 'Deep learning is the subset of machine learning methods based on neural networks with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]\n\nDeep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]\n\nEarly forms of neural networks were inspired by information processing and distributed communication nodes in biological systems, in particular the human brain. However, current neural networks do not intend to model th

In [46]:
quiz=response.get("quiz")

In [47]:
quiz=json.loads(quiz)

In [36]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [37]:
quiz_table_data

[{'MCQ': 'What is the subset of machine learning methods based on neural networks with representation learning?',
  'Choices': 'a: Supervised learning | b: Deep learning | c: Unsupervised learning | d: Reinforcement learning',
  'Correct': 'b'},
 {'MCQ': 'Which adjective refers to the use of multiple layers in the network in deep learning?',
  'Choices': 'a: Shallow | b: Wide | c: Deep | d: Narrow',
  'Correct': 'c'},
 {'MCQ': 'Which deep-learning architecture is commonly used for computer vision tasks?',
  'Choices': 'a: Deep belief networks | b: Recurrent neural networks | c: Convolutional neural networks | d: Transformers',
  'Correct': 'c'},
 {'MCQ': 'What field has deep learning NOT been applied to?',
  'Choices': 'a: Computer vision | b: Speech recognition | c: Agriculture | d: Natural language processing',
  'Correct': 'c'},
 {'MCQ': 'What were early forms of neural networks inspired by?',
  'Choices': 'a: Computer programming | b: Biological systems | c: Physics principles | d:

In [38]:
quiz=pd.DataFrame(quiz_table_data)

In [39]:
quiz

,MCQ,Choices,Correct
0,What is the subset of machine learning methods...,a: Supervised learning | b: Deep learning | c:...,b
1,Which adjective refers to the use of multiple ...,a: Shallow | b: Wide | c: Deep | d: Narrow,c
2,Which deep-learning architecture is commonly u...,a: Deep belief networks | b: Recurrent neural ...,c
3,What field has deep learning NOT been applied to?,a: Computer vision | b: Speech recognition | c...,c
4,What were early forms of neural networks inspi...,a: Computer programming | b: Biological system...,b


In [40]:
quiz.to_csv("machinelearning.csv",index=False)

In [43]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'06_16_2024_23_09_15'